Crawler para extração dos itens do carrinho da Atividade Prática.

# Imports

In [1]:
import requests, json, threading, sys, traceback
import pandas as pd

# Funções

In [2]:
def get_category(category: str) -> list:
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    payload = {
        "partner": "linx",
        "page": 1,
        "resultsPerPage": 300,
        "multiCategory": category,
        "sortBy": "relevance",
        "department": "ecom",
        "storeId": 461,
        "customerPlus": True
    }
    
    try:
        # Pega o número total de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        max_produtos = response.json()['totalProducts']
        payload['resultsPerPage'] = max_produtos

        # Pega a lista de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        
        lista_p = list(response.json()['products'])
        lista_cod = []

        for prod in lista_p:
            lista_cod.append(prod['id'])
            
    except:
        return []
    
    else:
        return lista_cod

In [3]:
def get_product(code: int, verbose: int = 0) -> dict:
    try:
        # Fazendo a requisição
        texto = requests.get(fr"https://www.paodeacucar.com/produto/{code}").text
        
        # Encontrando as posições
        inicio = texto.find('"props":') + len('"props":')
        fim = texto.find(',"page":"/produto"', inicio)

        # Extraindo a substring
        substring = texto[inicio:fim].strip()
        
        # Dicionario Geral
        dicionario = json.loads(substring)
        
        # Informações gerais do produto
        informacoes = dicionario['initialProps']['componentProps']['product']
        atributos = get_dictkey(informacoes, 'attributeGroups')
        
        # Dicionário com os resultados
        resultados = {    
        'id': informacoes['id'], 
        'nome': informacoes['name'], 
        'sku': informacoes['sku'], 
        'descricao': informacoes['description'],
        'imagem': f'https://static.paodeacucar.com{informacoes['thumbPath']}',
        'preco': get_dictkey(informacoes, 'currentPrice'),
        'marca': get_dictcode(atributos, 'Marca'),
        'alturaProduto': get_dictcode(atributos, 'alturaProduto'),
        'larguraProduto': get_dictcode(atributos, 'larguraProduto'),
        'pesoBruto': get_dictcode(atributos, 'pesoBruto'),
        'unidadePesoBruto': get_dictcode(atributos, 'unidadePesoBruto'),
        'profundidadeProduto': get_dictcode(atributos, 'profundidadeProduto')
        }
        
        assert(
            resultados['id'] == code and 
            resultados['preco'] != None and
            resultados['alturaProduto'] != None and 
            resultados['larguraProduto'] != None and 
            resultados['pesoBruto'] != None and 
            resultados['unidadePesoBruto'] != None and
            resultados['profundidadeProduto'] != None
        )

    except:
        if verbose >= 1:
            exc_info = sys.exc_info()
            e = ''.join(traceback.format_exception(*exc_info))
            del exc_info
            
            print(f"Erro ao extrair informações do produto {code}!\nErro:\n{e}")
        
        if verbose >= 2:     
            if 'informacoes' in locals():
                print(json.dumps(locals()['informacoes'], indent=2))
                
            if 'informacoes2' in locals():
                print(json.dumps(locals()['informacoes2'], indent=2))
        
        return None
    
    else:
        return resultados

In [4]:
def fetch_products(codigos, verbose: int = 0):
    resultados = []
    threads = []
    lock = threading.Lock()

    def thread_function(codigo, verbose):
        info = get_product(codigo, verbose)
        
        if info:
            with lock:
                resultados.append(info)

    for codigo in codigos:
        thread = threading.Thread(target=thread_function, args=(codigo, verbose,))
        threads.append(thread)
        thread.start()

    while any(thread.is_alive() for thread in threads): pass

    return resultados


In [5]:
def get_dictkey(dicionario, chave):
    # Função interna recursiva
    def busca_recursiva(d):
        if isinstance(d, dict):
            for k, v in d.items():
                if k == chave:
                    return v
                resultado = busca_recursiva(v)
                if resultado is not None:
                    return resultado
                
        elif isinstance(d, list):
            for item in d:
                resultado = busca_recursiva(item)
                if resultado is not None:
                    return resultado
        return None

    return busca_recursiva(dicionario)

In [6]:
def get_dictcode(d, code):
    # Se d for um dicionário, iteramos sobre suas chaves e valores
    if isinstance(d, dict):
        if d.get('code') == code:
            return d.get('value')  # Retorna o valor se a chave 'code' for encontrada com o valor especificado
        for value in d.values():
            result = get_dictcode(value, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    # Se d for uma lista, iteramos sobre os elementos da lista
    elif isinstance(d, list):
        for item in d:
            result = get_dictcode(item, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    return None  # Retorna None se o dicionário não for encontrado

In [7]:
def format_desc(x):
    for rep in ['<b>', '</b>', '<br>', '</br>', '<p>', '</p>', '\r', '\n', r'"""¿']:
        x = x.replace(rep, '')
    
    return x

# Constants

In [8]:
CATEGORIAS = [
    'alimentos', 
    'beleza-e-perfumaria',
    'bebidas', 
    'bebidas-alcoolicas-', 
    'limpeza', 
    'bebes-e-criancas', 
    'cuidados-pessoais', 
    'suplementos-alimentares', 
    'eventos-e-festas',
    'utensilios-e-descartaveis',
    'petshop', 
    'esporte-e-lazer',
    'casa-e-construcao',
    'floricultura-e-jardim'
]

# Code

In [9]:
map = []

for category in CATEGORIAS:
    print(f"Obtendo códigos de produtos da categoria {category}...")
    codigos = get_category(category)
    
    print(f"Obtendo informações dos produtos da categoria {category}...")
    # map[category] = fetch_products(codigos)
    for item in fetch_products(codigos, 0):
        map.append({**item, **{'categoria': category}})

map

Obtendo códigos de produtos da categoria alimentos...
Obtendo informações dos produtos da categoria alimentos...
Obtendo códigos de produtos da categoria beleza-e-perfumaria...
Obtendo informações dos produtos da categoria beleza-e-perfumaria...
Obtendo códigos de produtos da categoria bebidas...
Obtendo informações dos produtos da categoria bebidas...
Obtendo códigos de produtos da categoria bebidas-alcoolicas-...
Obtendo informações dos produtos da categoria bebidas-alcoolicas-...
Obtendo códigos de produtos da categoria limpeza...
Obtendo informações dos produtos da categoria limpeza...
Obtendo códigos de produtos da categoria bebes-e-criancas...
Obtendo informações dos produtos da categoria bebes-e-criancas...
Obtendo códigos de produtos da categoria cuidados-pessoais...
Obtendo informações dos produtos da categoria cuidados-pessoais...
Obtendo códigos de produtos da categoria suplementos-alimentares...
Obtendo informações dos produtos da categoria suplementos-alimentares...
Obtend

[{'id': 65885,
  'nome': 'Filé de Peito de Frango Zip SADIA Pacote 1kg',
  'sku': '1799316',
  'descricao': '<b>Ingrediente:</b><br>Filé de Frango.',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/999/28039999.png',
  'preco': 29.9,
  'marca': 'Sadia',
  'alturaProduto': '22',
  'larguraProduto': '14',
  'pesoBruto': '1.07',
  'unidadePesoBruto': 'KG',
  'profundidadeProduto': '7',
  'categoria': 'alimentos'},
 {'id': 9461,
  'nome': 'Feijão Carioca Tipo 1 CAMIL Pacote 1kg',
  'sku': '0468084',
  'descricao': 'FEIJAO CARIOCA CAMIL T1 1KG',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/29/10519029.jpeg',
  'preco': 7.99,
  'marca': 'Camil',
  'alturaProduto': '22.5',
  'larguraProduto': '14.5',
  'pesoBruto': '1.012',
  'unidadePesoBruto': 'KG',
  'profundidadeProduto': '4',
  'categoria': 'alimentos'},
 {'id': 339743,
  'nome': 'Queijo Mussarela Fatiado Président 150g',
  'sku': '1102447',
  'descricao': '',
  'imagem': 'https://static.paodeacucar.com/img/uploads

In [10]:
df = pd.concat([pd.DataFrame([item]) for item in map], ignore_index=True, sort=False, axis='index')
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,unidadePesoBruto,profundidadeProduto,categoria
0,65885,Filé de Peito de Frango Zip SADIA Pacote 1kg,1799316,<b>Ingrediente:</b><br>Filé de Frango.,https://static.paodeacucar.com/img/uploads/1/9...,29.90,Sadia,22,14,1.07,KG,7,alimentos
1,9461,Feijão Carioca Tipo 1 CAMIL Pacote 1kg,0468084,FEIJAO CARIOCA CAMIL T1 1KG,https://static.paodeacucar.com/img/uploads/1/2...,7.99,Camil,22.5,14.5,1.012,KG,4,alimentos
2,339743,Queijo Mussarela Fatiado Président 150g,1102447,,https://static.paodeacucar.com/img/uploads/1/2...,10.90,President,18.8,13.2,164.2,G,1.7,alimentos
3,99080,Arroz Agulhinha Tipo 1 CAMIL Pacote 1kg,0300438,<b>Ingrediente:</b><br> Arroz.</b><br><br><b>N...,https://static.paodeacucar.com/img/uploads/1/7...,7.49,Camil,22.5,14.5,1.065,KG,4,alimentos
4,72624,Mamão Papaya Granel 900g (1 Unid),0259163,"Rico em vitamina A, potássio e vitamina C<Br><...",https://static.paodeacucar.com/img/uploads/1/6...,11.61,Bandeiras,1,1,1,KG,1,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,1614834,Acendedor Gel para Carvão e Lenha 80° INPM Qua...,1379084,,https://static.paodeacucar.com/img/uploads/1/1...,13.79,Qualitá,21,7.85,455,G,4.9,casa-e-construcao
717,433571,Gel Acendedor FIAT LUX 420g,1182325,,https://static.paodeacucar.com/img/uploads/1/3...,16.99,Fiat Lux,19.9,6.6,483.9,G,6.6,casa-e-construcao
718,1420972,Vaso Cachepot Siena 16x14cm Rosa Coral,1342365,,https://static.paodeacucar.com/img/uploads/1/5...,49.99,West Garden,14.8,16,100,G,16,floricultura-e-jardim
719,88856,Pulverizador manual spray West - 500 ml,0198530,,https://static.paodeacucar.com/img/uploads/1/6...,19.99,West Garden,36,15.1,100,G,8,floricultura-e-jardim


In [11]:
df['descricao'] = df['descricao'].apply(format_desc)
df['descricao'] = df['descricao'].fillna('Sem descrição')
df['pesoBruto'] = df['pesoBruto'].astype(float)
df['pesoBruto'] = df.apply(lambda x: x['pesoBruto'] if x['unidadePesoBruto'] == 'G' else x['pesoBruto'] * 1000, axis=1)
df.drop(columns=['unidadePesoBruto'], inplace=True)
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,profundidadeProduto,categoria
0,65885,Filé de Peito de Frango Zip SADIA Pacote 1kg,1799316,Ingrediente:Filé de Frango.,https://static.paodeacucar.com/img/uploads/1/9...,29.90,Sadia,22,14,1070.0,7,alimentos
1,9461,Feijão Carioca Tipo 1 CAMIL Pacote 1kg,0468084,FEIJAO CARIOCA CAMIL T1 1KG,https://static.paodeacucar.com/img/uploads/1/2...,7.99,Camil,22.5,14.5,1012.0,4,alimentos
2,339743,Queijo Mussarela Fatiado Président 150g,1102447,,https://static.paodeacucar.com/img/uploads/1/2...,10.90,President,18.8,13.2,164.2,1.7,alimentos
3,99080,Arroz Agulhinha Tipo 1 CAMIL Pacote 1kg,0300438,Ingrediente: Arroz.NÃO CONTÉM GLÚTEN.,https://static.paodeacucar.com/img/uploads/1/7...,7.49,Camil,22.5,14.5,1065.0,4,alimentos
4,72624,Mamão Papaya Granel 900g (1 Unid),0259163,"Rico em vitamina A, potássio e vitamina C<Br>D...",https://static.paodeacucar.com/img/uploads/1/6...,11.61,Bandeiras,1,1,1000.0,1,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...
716,1614834,Acendedor Gel para Carvão e Lenha 80° INPM Qua...,1379084,,https://static.paodeacucar.com/img/uploads/1/1...,13.79,Qualitá,21,7.85,455.0,4.9,casa-e-construcao
717,433571,Gel Acendedor FIAT LUX 420g,1182325,,https://static.paodeacucar.com/img/uploads/1/3...,16.99,Fiat Lux,19.9,6.6,483.9,6.6,casa-e-construcao
718,1420972,Vaso Cachepot Siena 16x14cm Rosa Coral,1342365,,https://static.paodeacucar.com/img/uploads/1/5...,49.99,West Garden,14.8,16,100.0,16,floricultura-e-jardim
719,88856,Pulverizador manual spray West - 500 ml,0198530,,https://static.paodeacucar.com/img/uploads/1/6...,19.99,West Garden,36,15.1,100.0,8,floricultura-e-jardim


In [13]:
PATH_TO_SAVE = '..\data\produtos.csv'

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Arklok\AppData\Local\Temp\ipykernel_15972\3786087928.py:1: SyntaxWarning: invalid escape sequence '\d'
  PATH_TO_SAVE = '..\data\produtos.csv'


In [ ]:
old_df = pd.read_csv(PATH_TO_SAVE, sep='|', encoding='utf-8')

In [ ]:
df = df[~df['id'].isin(old_df['id'])]

In [ ]:
df = pd.concat([old_df, df], ignore_index=True, sort=False, axis='index')

In [12]:
df.to_csv(r'..\data\produtos.csv', index=False, sep='|', encoding='utf-8')